In [173]:
import pandas as pd
from datetime import datetime
import requests

In [174]:
start_date = '2023-01-01'
end_date = '2024-05-27'

In [175]:
url = f'https://api.energy-charts.info/price?bzn=SI&start={start_date}&end={end_date}'
response = requests.get(url)

data = response.json()

In [176]:
print('Number of price items:', len(data['price']))
print('Number of unix time items:', len(data['unix_seconds']))

if len(data['price']) == len(data['unix_seconds']):
    print('Lengths are matching the expected values')

In [177]:
print('Response data from:', datetime.fromtimestamp(data['unix_seconds'][0]).strftime('%Y-%m-%d %H:%M:%S'))
print('Response data to:', datetime.fromtimestamp(data['unix_seconds'][-1]).strftime('%Y-%m-%d %H:%M:%S'))

In [178]:
daily_price_data = []
for i in range(len(data['price'])):
    timestamp = datetime.fromtimestamp(data['unix_seconds'][i]).strftime('%Y-%m-%dT%H:%M:%S')
    price = data['price'][i]
    
    new_data = {'date': timestamp, 'price': price}
    daily_price_data.append(new_data)

In [179]:
df = pd.DataFrame(daily_price_data)
df.head(48)

In [180]:
weather_url = f'https://archive-api.open-meteo.com/v1/archive?latitude=46.0833&longitude=15&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,cloud_cover,wind_speed_10m&timezone=Europe%2FBerlin'
response = requests.get(weather_url)

weather_data = response.json()

In [181]:
daily_weather_data = []
for i in range(len(weather_data['hourly']['time'])):
    temperature = weather_data['hourly']['temperature_2m'][i]
    humidity = weather_data['hourly']['relative_humidity_2m'][i]
    precipitation = weather_data['hourly']['precipitation'][i]
    cloud_cover = weather_data['hourly']['cloud_cover'][i]
    wind_speed = weather_data['hourly']['wind_speed_10m'][i]
    
    new_data = {'date': weather_data['hourly']['time'][i], 'temperature': temperature, 'humidity': humidity, 'precipitation': precipitation, 'cloud_cover': cloud_cover, 'wind_speed': wind_speed}
    daily_weather_data.append(new_data)

In [182]:
df_weather = pd.DataFrame(daily_weather_data)
df_weather.head(48)

In [183]:
print(df_weather.shape)
print(df.shape)

In [184]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

In [185]:
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather.dtypes

In [186]:
df.isnull().sum()

In [187]:
df_weather.isnull().sum()

In [188]:
df_merged = pd.merge(df, df_weather, on='date')
# df_merged.drop('date', axis=1, inplace=True)
df_merged.head()

In [189]:
df_merged.to_csv('../data/processed/price_data.csv', index=False)